# 🧠 Neural Proof Assistant - Training Notebook

This notebook trains a proof technique classifier using:
1. **Weak Supervision** - Labeling functions generate training data
2. **Sentence Transformers** - Pre-trained embeddings for semantic understanding
3. **Classifier** - Learns to generalize beyond the rules

**Output:** `classifier.pkl` and `label_encoder.pkl` to download and use in your FastAPI backend.

## Step 0: Install Dependencies

In [2]:
%pip install --upgrade torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.3/807.3 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [torchaudio]2 [torchaudio]

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install sentence-transformers scikit-learn pandas numpy --upgrade -q


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Step 1: Define Labeling Functions

These are the same labeling functions from your FastAPI backend. They generate weak labels for training.

In [4]:
import re
from enum import IntEnum
from typing import List

class ProofTechnique(IntEnum):
    ABSTAIN = -1
    DIRECT = 0
    CONTRADICTION = 1
    INDUCTION = 2
    CONTRAPOSITIVE = 3
    CONSTRUCTION = 4
    CASES = 5
    EXHAUSTION = 6

TECHNIQUE_NAMES = {
    0: "Direct Proof",
    1: "Proof by Contradiction",
    2: "Mathematical Induction",
    3: "Contrapositive",
    4: "Proof by Construction",
    5: "Proof by Cases",
    6: "Proof by Exhaustion"
}

class LabelingFunction:
    def __init__(self, name: str, weight: float = 1.0):
        self.name = name
        self.weight = weight

    def __call__(self, text: str) -> int:
        raise NotImplementedError


class KeywordLF(LabelingFunction):
    def __init__(self, name: str, keywords: List[str], label: int,
                 negative_keywords: List[str] = None, weight: float = 1.0):
        super().__init__(name, weight)
        self.keywords = [k.lower() for k in keywords]
        self.negative_keywords = [k.lower() for k in (negative_keywords or [])]
        self.label = label

    def __call__(self, text: str) -> int:
        text_lower = text.lower()
        for neg in self.negative_keywords:
            if neg in text_lower:
                return ProofTechnique.ABSTAIN
        if any(kw in text_lower for kw in self.keywords):
            return self.label
        return ProofTechnique.ABSTAIN


class PatternLF(LabelingFunction):
    def __init__(self, name: str, patterns: List[str], label: int, weight: float = 1.0):
        super().__init__(name, weight)
        self.patterns = [re.compile(p, re.IGNORECASE) for p in patterns]
        self.label = label

    def __call__(self, text: str) -> int:
        for pattern in self.patterns:
            if pattern.search(text):
                return self.label
        return ProofTechnique.ABSTAIN


class StructuralLF(LabelingFunction):
    def __init__(self, name: str, required_sections: List[str], label: int, weight: float = 1.0):
        super().__init__(name, weight)
        self.required_sections = [s.lower() for s in required_sections]
        self.label = label

    def __call__(self, text: str) -> int:
        text_lower = text.lower()
        found_count = sum(1 for section in self.required_sections if section in text_lower)
        if found_count >= len(self.required_sections) * 0.66:
            return self.label
        return ProofTechnique.ABSTAIN

print("✅ Labeling function classes defined")

✅ Labeling function classes defined


In [5]:
# Define all labeling functions
LABELING_FUNCTIONS = [
    # === DIRECT PROOF ===
    KeywordLF("direct_keywords", ["we have", "it follows that", "thus we see", "clearly"],
              ProofTechnique.DIRECT,
              negative_keywords=["contradiction", "contradicts", "suppose not", "induction", "base case"],
              weight=0.8),
    PatternLF("direct_pattern", [r"assume\s+.{5,50}\.\s*then", r"let\s+.{3,30}\s+be\s+.{3,30}\.\s*then"],
              ProofTechnique.DIRECT, weight=1.0),

    # === CONTRADICTION ===
    KeywordLF("contradiction_explicit", ["contradiction", "contradicts", "contradicting"],
              ProofTechnique.CONTRADICTION, weight=1.5),
    KeywordLF("contradiction_absurd", ["absurd", "impossible", "cannot be"],
              ProofTechnique.CONTRADICTION, weight=1.2),
    PatternLF("contradiction_pattern_suppose",
              [r"suppose\s+(not|.*\s+not\s+)", r"assume\s+(for\s+)?contradiction",
               r"assume\s+the\s+contrary", r"for\s+the\s+sake\s+of\s+contradiction",
               r"suppose.*rational", r"suppose.*is\s+rational"],
              ProofTechnique.CONTRADICTION, weight=1.3),
    PatternLF("contradiction_pattern_derive",
              [r"this\s+(is\s+a\s+)?contradiction", r"which\s+contradicts",
               r"contradicting\s+(our|the)\s+(assumption|hypothesis)", r"this\s+is\s+absurd",
               r"but\s+this\s+contradicts", r"this\s+contradicts"],
              ProofTechnique.CONTRADICTION, weight=1.6),

    # === INDUCTION ===
    KeywordLF("induction_explicit", ["induction", "inductive"],
              ProofTechnique.INDUCTION, weight=1.3),
    StructuralLF("induction_structure", ["base case", "inductive step", "inductive hypothesis"],
                 ProofTechnique.INDUCTION, weight=1.5),
    PatternLF("induction_pattern_base",
              [r"base\s+case.*n\s*=\s*[01]", r"for\s+n\s*=\s*[01].*holds", r"when\s+n\s*=\s*[01]"],
              ProofTechnique.INDUCTION, weight=1.2),
    PatternLF("induction_pattern_step",
              [r"inductive\s+step", r"assume\s+(true\s+)?for\s+n\s*=\s*k",
               r"by\s+(the\s+)?inductive\s+hypothesis"],
              ProofTechnique.INDUCTION, weight=1.4),

    # === CONTRAPOSITIVE ===
    KeywordLF("contrapositive_explicit", ["contrapositive", "contrapositively"],
              ProofTechnique.CONTRAPOSITIVE, weight=1.4),
    PatternLF("contrapositive_pattern",
              [r"prove\s+the\s+contrapositive", r"by\s+contrapositive",
               r"if\s+not.*then\s+not"],
              ProofTechnique.CONTRAPOSITIVE, weight=1.3),

    # === CONSTRUCTION ===
    KeywordLF("construction_explicit", ["construct", "constructive", "exhibit"],
              ProofTechnique.CONSTRUCTION,
              negative_keywords=["construct a contradiction"], weight=1.1),
    PatternLF("construction_pattern",
              [r"we\s+construct", r"we\s+exhibit", r"consider\s+the\s+(following\s+)?example",
               r"such\s+a.{1,20}exists.*namely"],
              ProofTechnique.CONSTRUCTION, weight=1.0),

    # === CASES ===
    KeywordLF("cases_explicit", ["case 1", "case 2", "case i", "case ii"],
              ProofTechnique.CASES, weight=1.3),
    PatternLF("cases_pattern",
              [r"we\s+(consider|divide)\s+(into\s+)?(two|three|several)\s+cases",
               r"there\s+are\s+(two|three|several)\s+cases",
               r"case\s+\d+\s*:", r"in\s+(the\s+)?(first|second)\s+case"],
              ProofTechnique.CASES, weight=1.2),
    PatternLF("cases_structure", [r"either.*or", r"suppose.*is\s+(even|odd)"],
              ProofTechnique.CASES, weight=0.9),

    # === EXHAUSTION ===
    PatternLF("exhaustion_pattern",
              [r"check\s+(each|all|every)", r"by\s+exhaustion",
               r"checking\s+all\s+(possible\s+)?cases", r"we\s+enumerate"],
              ProofTechnique.EXHAUSTION, weight=1.2),
    KeywordLF("exhaustion_keywords", ["exhaustively", "finite check", "brute force"],
              ProofTechnique.EXHAUSTION, weight=1.1),
]

print(f"✅ Defined {len(LABELING_FUNCTIONS)} labeling functions")

✅ Defined 19 labeling functions


## Step 2: Create Training Dataset

We'll create sample proofs and use the labeling functions to generate weak labels.

In [6]:
import numpy as np

# Sample proofs for each technique (you can add more!)
SAMPLE_PROOFS = {
    ProofTechnique.CONTRADICTION: [
        "Suppose √2 is rational. Then √2 = a/b where a, b have no common factors. Squaring, a² = 2b². This means a is even. Write a = 2k. Then b² = 2k², so b is also even. This contradicts our assumption. Therefore √2 is irrational.",
        "Assume for contradiction that there are finitely many primes p₁, p₂, ..., pₙ. Consider N = p₁p₂...pₙ + 1. N is not divisible by any pᵢ. This is a contradiction since every integer > 1 has a prime factor.",
        "Suppose the square root of 3 is rational. Then √3 = a/b in lowest terms. So 3b² = a², meaning a² is divisible by 3. Thus a is divisible by 3. Write a = 3k. Then 3b² = 9k², so b² = 3k². But then b is also divisible by 3, contradicting lowest terms.",
        "Assume there exists a smallest positive rational number r. Consider r/2. This is positive and rational, but smaller than r. This contradicts our assumption that r was smallest.",
        "Suppose √5 is rational, so √5 = p/q with gcd(p,q) = 1. Then 5q² = p². Thus 5 divides p², so 5 divides p. Let p = 5k. Then 5q² = 25k², giving q² = 5k². So 5 divides q. This contradicts gcd(p,q) = 1.",
        "For the sake of contradiction, assume log₂(3) is rational. Then log₂(3) = p/q for integers p, q. So 2^(p/q) = 3, meaning 2^p = 3^q. But 2^p is even and 3^q is odd. This is a contradiction.",
        "Suppose there is a largest prime number P. Consider N = P! + 1. For any prime p ≤ P, we have p | P!, so p ∤ N. Thus N has a prime factor greater than P. This contradicts P being largest.",
        "Assume √6 is rational. Write √6 = a/b in lowest terms. Then 6b² = a², so a² is even, hence a is even. Let a = 2k. Then 6b² = 4k², so 3b² = 2k². Thus b² is even, so b is even. This contradicts a/b being in lowest terms.",
        "Suppose, seeking a contradiction, that there exists an integer that is both even and odd. Then n = 2k and n = 2m + 1 for some integers k, m. So 2k = 2m + 1, giving 2(k-m) = 1. But 2 does not divide 1. Contradiction.",
        "Assume for contradiction that 0 = 1. Then for any x, we have x = x·1 = x·0 = 0. But this means all numbers equal 0, which is absurd.",
        "We proceed by contradiction. Suppose there is a rational solution to x² = 2. Let x = p/q in lowest terms. Then p² = 2q². So p² is even, meaning p is even. Write p = 2r. Then 4r² = 2q², so q² = 2r². Thus q is even. But this contradicts p/q being in lowest terms.",
        "Suppose that the set of real numbers is countable. By Cantor's diagonal argument, we can construct a real number not in our list. This is a contradiction, so the reals are uncountable.",
    ],

    ProofTechnique.INDUCTION: [
        "We prove by induction that 1 + 2 + ... + n = n(n+1)/2. Base case: For n = 1, we have 1 = 1(2)/2 = 1. ✓ Inductive step: Assume true for n = k. Then 1 + 2 + ... + k + (k+1) = k(k+1)/2 + (k+1) = (k+1)(k+2)/2. By induction, the formula holds for all n.",
        "Proof by induction on n. Base case: When n = 0, 2⁰ = 1 ≥ 1 = 0 + 1. Inductive step: Assume 2^k ≥ k + 1. Then 2^(k+1) = 2·2^k ≥ 2(k+1) = 2k + 2 ≥ k + 2 = (k+1) + 1. By the principle of mathematical induction, 2^n ≥ n + 1 for all n ≥ 0.",
        "We use strong induction. Base case: F₁ = 1 < 2 = 2¹. Inductive step: Assume Fₖ < 2^k for all k < n. Then Fₙ = Fₙ₋₁ + Fₙ₋₂ < 2^(n-1) + 2^(n-2) < 2^(n-1) + 2^(n-1) = 2^n. By strong induction, Fₙ < 2^n for all n.",
        "Proof by induction. Base case: For n = 1, we have 1² = 1 = 1(2)(3)/6. ✓ Inductive step: Suppose 1² + 2² + ... + k² = k(k+1)(2k+1)/6. Adding (k+1)², we get k(k+1)(2k+1)/6 + (k+1)² = (k+1)(k+2)(2k+3)/6. Done by induction.",
        "We prove n! > 2^n for n ≥ 4 by induction. Base case: 4! = 24 > 16 = 2⁴. Inductive step: Assume k! > 2^k for some k ≥ 4. Then (k+1)! = (k+1)·k! > (k+1)·2^k > 2·2^k = 2^(k+1). By mathematical induction, the result holds.",
        "By induction on n. Base case: n = 1 gives 1 = 1. Inductive hypothesis: Assume the formula holds for n = k. Inductive step: For n = k + 1, we have... By the inductive hypothesis, this equals... Therefore by induction, the formula holds for all positive integers.",
        "We prove by induction that n³ - n is divisible by 6. Base case: When n = 1, n³ - n = 0, which is divisible by 6. Inductive step: Assume k³ - k ≡ 0 (mod 6). Then (k+1)³ - (k+1) = k³ + 3k² + 3k + 1 - k - 1 = (k³ - k) + 3k(k+1). Since k(k+1) is even, 3k(k+1) ≡ 0 (mod 6). By induction, done.",
        "Proof by induction. Base case: For n = 0, the empty product equals 1. Inductive step: Assume the result for n = k. For n = k + 1, by the inductive hypothesis... This completes the induction.",
        "We use induction on the number of elements. Base case: A set with 0 elements has 2⁰ = 1 subset (the empty set). Inductive step: Assume a set with k elements has 2^k subsets. Adding one element doubles the subsets (each old subset with or without the new element). So 2·2^k = 2^(k+1) subsets.",
        "By induction. Base case: n = 1 satisfies the inequality. Inductive hypothesis: Suppose the inequality holds for n = k. Inductive step: We must show it holds for n = k + 1. Using the inductive hypothesis... Therefore by mathematical induction, the inequality holds for all n ≥ 1.",
    ],

    ProofTechnique.CASES: [
        "We prove n² has the same parity as n. Case 1: Suppose n is even. Then n = 2k, so n² = 4k² = 2(2k²), which is even. Case 2: Suppose n is odd. Then n = 2k + 1, so n² = 4k² + 4k + 1 = 2(2k² + 2k) + 1, which is odd.",
        "We consider two cases. Case 1: If x ≥ 0, then |x| = x ≥ 0. Case 2: If x < 0, then |x| = -x > 0. In both cases, |x| ≥ 0.",
        "There are three cases to consider. Case i: n ≡ 0 (mod 3). Case ii: n ≡ 1 (mod 3). Case iii: n ≡ 2 (mod 3). In each case, we verify the claim holds.",
        "We divide into cases based on the sign of x. First case: x > 0. Second case: x = 0. Third case: x < 0. Combining all cases, the result follows.",
        "Proof by cases. Either n is divisible by 2, or n is not divisible by 2. In the first case... In the second case... Therefore the result holds in all cases.",
        "We split into two cases depending on whether n is even or odd. Case 1: n = 2m for some integer m. Then... Case 2: n = 2m + 1 for some integer m. Then... This covers all possibilities.",
        "Consider the cases separately. Case 1: Suppose a ≤ b. Then max(a,b) = b. Case 2: Suppose a > b. Then max(a,b) = a. In either case, max(a,b) ≥ a and max(a,b) ≥ b.",
        "We analyze by cases. In the first case, assume x² < 1. In the second case, assume x² = 1. In the third case, assume x² > 1. Each case leads to the desired conclusion.",
        "There are two cases. If n is prime, then... If n is composite, then... Either way, the statement holds.",
        "We proceed case by case. Case (a): the denominator is positive. Case (b): the denominator is negative. Note the denominator cannot be zero. Both cases yield the result.",
    ],

    ProofTechnique.CONSTRUCTION: [
        "To show there exists an even prime, we exhibit 2. Indeed, 2 has exactly two divisors: 1 and 2. And 2 = 2·1 is even. Thus we have constructed an even prime.",
        "We construct an explicit example. Let f(x) = x². Then f is continuous everywhere but differentiable nowhere... Actually, let f(x) = |x|. At x = 0, f is continuous but not differentiable.",
        "To prove existence, we construct: take n = 6. Then n = 2·3 = 1 + 2 + 3, showing 6 is both a product and sum of consecutive integers.",
        "We exhibit a concrete example. Consider the matrix A = [[1, 2], [0, 1]]. Then A² = [[1, 4], [0, 1]] ≠ I, showing A is not an involution but has determinant 1.",
        "Such a number exists: namely, take x = √2 + √3. We verify that x is irrational and that x² = 5 + 2√6 is also irrational.",
        "We construct the required function explicitly. Define f: ℝ → ℝ by f(x) = x³. This function is bijective, continuous, and satisfies f(-x) = -f(x).",
        "To show a solution exists, we construct one. Let x = 2, y = 3, z = 1. Then x² + y² = 4 + 9 = 13 and z² + 12 = 1 + 12 = 13. So (2, 3, 1) is a solution.",
        "The following example demonstrates the claim. Take the sequence aₙ = (-1)ⁿ/n. This sequence converges to 0 but is not monotonic.",
        "We exhibit an explicit counterexample. Let A = {1, 2} and B = {2, 3}. Then A ∪ B = {1, 2, 3} has 3 elements, but |A| + |B| = 4. So |A ∪ B| ≠ |A| + |B| in general.",
        "Consider the polynomial p(x) = x² - 2. By construction, p(√2) = 2 - 2 = 0, so √2 is a root of p(x).",
    ],

    ProofTechnique.CONTRAPOSITIVE: [
        "We prove the contrapositive: if n is odd, then n² is odd. Suppose n = 2k + 1. Then n² = 4k² + 4k + 1 = 2(2k² + 2k) + 1, which is odd. By contrapositive, if n² is even, then n is even.",
        "Instead of proving P → Q directly, we prove the contrapositive ¬Q → ¬P. Assume ¬Q. Then... Therefore ¬P. By contrapositive, P → Q.",
        "Proof by contrapositive. We show: if n² is not divisible by 3, then n is not divisible by 3. Suppose 3 ∤ n². Then n² ≡ 1 or 4 (mod 9), so n ≢ 0 (mod 3). Contrapositively, 3|n implies 3|n².",
        "We prove the contrapositive statement. If f is not continuous, then f is not differentiable. This is equivalent to: if f is differentiable, then f is continuous.",
        "The contrapositive is: if xy is odd, then both x and y are odd. Suppose xy is odd. If x were even, xy would be even. So x must be odd. Similarly y is odd. By contrapositive, x or y even implies xy even.",
        "Rather than prove directly, we prove the contrapositive. Assume the conclusion fails. Then... which implies the hypothesis fails. By contrapositive, the original implication holds.",
        "Proof via contrapositive. Contrapositive statement: if not B, then not A. Assuming ¬B, we derive... hence ¬A. The contrapositive being true, the original statement A → B follows.",
        "We establish the contrapositive. If n is not prime, then 2ⁿ - 1 is not prime. Let n = ab with 1 < a, b < n. Then 2ⁿ - 1 = (2^a - 1)(2^a(b-1) + ... + 1). By contrapositive, 2ⁿ - 1 prime implies n prime.",
    ],

    ProofTechnique.DIRECT: [
        "Let n be an even integer. Then n = 2k for some integer k. Therefore n² = 4k² = 2(2k²), which is clearly even. Thus the square of an even number is even.",
        "Assume a | b and b | c. Then b = am and c = bn for some integers m, n. Therefore c = amn = a(mn), so a | c. This proves transitivity of divisibility.",
        "Let x and y be rational numbers. Then x = a/b and y = c/d for integers a, b, c, d with b, d ≠ 0. We have x + y = (ad + bc)/(bd). Since ad + bc and bd are integers and bd ≠ 0, x + y is rational.",
        "Suppose f and g are continuous at a. For any ε > 0, there exist δ₁, δ₂ > 0 such that... Taking δ = min(δ₁, δ₂), it follows that f + g is continuous at a.",
        "Let A ⊆ B and B ⊆ C. Take any x ∈ A. Since A ⊆ B, we have x ∈ B. Since B ⊆ C, we have x ∈ C. Therefore A ⊆ C.",
        "Assume n is divisible by 6. Then n = 6k for some integer k. We have n = 6k = 2(3k), so 2 | n. Also n = 6k = 3(2k), so 3 | n. Thus n is divisible by both 2 and 3.",
        "Let f be differentiable at a. By definition, f'(a) = lim[h→0] (f(a+h) - f(a))/h exists. Therefore f(a+h) - f(a) → 0 as h → 0, which means f is continuous at a.",
        "We prove directly that the sum of two odd numbers is even. Let m = 2j + 1 and n = 2k + 1 be odd. Then m + n = 2j + 1 + 2k + 1 = 2(j + k + 1), which is even.",
    ],

    ProofTechnique.EXHAUSTION: [
        "We verify by exhaustion. For n = 1: 1² = 1 ✓. For n = 2: 2² = 4 ✓. For n = 3: 3² = 9 ✓. For n = 4: 4² = 16 ✓. Having checked all cases, the claim holds.",
        "By exhaustive case checking: 0² mod 4 = 0, 1² mod 4 = 1, 2² mod 4 = 0, 3² mod 4 = 1. So squares are ≡ 0 or 1 (mod 4).",
        "Since there are only finitely many possibilities, we enumerate all of them. Checking each one... Therefore the claim holds.",
        "We verify the identity for all 8 possible combinations of signs. (+,+,+): ... (+,+,-): ... And so on. By exhaustion, the identity holds in all cases.",
        "The only divisors of 12 are 1, 2, 3, 4, 6, 12. Checking each: 1+2+3+4+6 = 16 > 12. So 12 is abundant.",
        "We check all n from 1 to 100 by brute force computation. The only solutions are n = 1, 8, and 17.",
        "Since n ∈ {1, 2, 3, 4, 5}, we check each case exhaustively. For n=1: valid. For n=2: valid. For n=3: invalid. For n=4: valid. For n=5: invalid. Thus n ∈ {1, 2, 4}.",
        "Enumerate all permutations of {1, 2, 3}: (1,2,3), (1,3,2), (2,1,3), (2,3,1), (3,1,2), (3,2,1). Exactly 2 of these satisfy the constraint.",
    ],
}

print("✅ Sample proofs defined")
for technique, proofs in SAMPLE_PROOFS.items():
    print(f"   {TECHNIQUE_NAMES[technique]}: {len(proofs)} examples")

✅ Sample proofs defined
   Proof by Contradiction: 12 examples
   Mathematical Induction: 10 examples
   Proof by Cases: 10 examples
   Proof by Construction: 10 examples
   Contrapositive: 8 examples
   Direct Proof: 8 examples
   Proof by Exhaustion: 8 examples


In [7]:
def apply_labeling_functions(text: str) -> int:
    """
    Apply all labeling functions and return majority vote.
    Returns ABSTAIN (-1) if no clear winner.
    """
    votes = {}
    total_weight = 0

    for lf in LABELING_FUNCTIONS:
        label = lf(text)
        if label != ProofTechnique.ABSTAIN:
            votes[label] = votes.get(label, 0) + lf.weight
            total_weight += lf.weight

    if not votes:
        return ProofTechnique.ABSTAIN

    return max(votes.keys(), key=lambda k: votes[k])


# Create training data
train_texts = []
train_labels = []

for technique, proofs in SAMPLE_PROOFS.items():
    for proof in proofs:
        # Use the ground truth label (from our curated examples)
        train_texts.append(proof)
        train_labels.append(int(technique))

print(f"✅ Created {len(train_texts)} training examples")
print(f"   Label distribution: {dict(zip(*np.unique(train_labels, return_counts=True)))}")

✅ Created 66 training examples
   Label distribution: {np.int64(0): np.int64(8), np.int64(1): np.int64(12), np.int64(2): np.int64(10), np.int64(3): np.int64(8), np.int64(4): np.int64(10), np.int64(5): np.int64(10), np.int64(6): np.int64(8)}


## Step 3: Generate Embeddings

Convert proof text → 384-dimensional vectors using a pre-trained sentence transformer.

In [8]:
from sentence_transformers import SentenceTransformer

print("Loading sentence transformer model...")
encoder = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Model loaded")

print("\nGenerating embeddings...")
X_embeddings = encoder.encode(train_texts, show_progress_bar=True)
print(f"✅ Generated embeddings with shape: {X_embeddings.shape}")

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading sentence transformer model...
✅ Model loaded

Generating embeddings...


Batches: 100%|██████████| 3/3 [00:04<00:00,  1.62s/it]

✅ Generated embeddings with shape: (66, 384)


## Step 4: Train Classifier

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(train_labels)

print("Training classifiers...\n")

# Try both Logistic Regression and MLP
classifiers = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "MLP (Neural Network)": MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
}

best_clf = None
best_score = 0
best_name = ""

for name, clf in classifiers.items():
    scores = cross_val_score(clf, X_embeddings, y_encoded, cv=5)
    mean_score = scores.mean()
    print(f"{name}:")
    print(f"   Cross-val accuracy: {mean_score:.1%} (+/- {scores.std()*2:.1%})")

    if mean_score > best_score:
        best_score = mean_score
        best_clf = clf
        best_name = name

print(f"\n✅ Best model: {best_name} ({best_score:.1%} accuracy)")

# Train final model on all data
print(f"\nTraining final {best_name} on all data...")
best_clf.fit(X_embeddings, y_encoded)
print("✅ Training complete")

Training classifiers...

Logistic Regression:
   Cross-val accuracy: 66.7% (+/- 18.0%)
MLP (Neural Network):
   Cross-val accuracy: 77.4% (+/- 27.1%)

✅ Best model: MLP (Neural Network) (77.4% accuracy)

Training final MLP (Neural Network) on all data...
✅ Training complete


## Step 5: Test the Model

In [10]:
# Test on new proofs
test_proofs = [
    "Suppose for contradiction that √7 is rational. Then √7 = a/b in lowest terms. So 7b² = a², meaning 7|a. Let a = 7k. Then 7b² = 49k², so b² = 7k², meaning 7|b. But this contradicts a/b being in lowest terms. Hence √7 is irrational.",
    "We prove by induction. Base case: For n=0, the sum is 0 = 0(0+1)/2. Inductive step: Assume true for k. Then 0+1+...+k+(k+1) = k(k+1)/2 + (k+1) = (k+1)(k+2)/2. By induction, done.",
    "Consider two cases. Case 1: If x > 0, then |x| = x, so |x|/x = 1. Case 2: If x < 0, then |x| = -x, so |x|/x = -1. Thus |x|/x = ±1 for all x ≠ 0.",
]

print("Testing on new proofs:\n")
print("=" * 70)

for proof in test_proofs:
    embedding = encoder.encode([proof])
    pred_encoded = best_clf.predict(embedding)[0]
    pred_label = label_encoder.inverse_transform([pred_encoded])[0]
    proba = best_clf.predict_proba(embedding)[0]
    confidence = proba[pred_encoded]

    print(f"Proof: {proof[:80]}...")
    print(f"Predicted: {TECHNIQUE_NAMES[pred_label]}")
    print(f"Confidence: {confidence:.1%}")
    print("-" * 70)

Testing on new proofs:

Proof: Suppose for contradiction that √7 is rational. Then √7 = a/b in lowest terms. So...
Predicted: Proof by Contradiction
Confidence: 99.6%
----------------------------------------------------------------------
Proof: We prove by induction. Base case: For n=0, the sum is 0 = 0(0+1)/2. Inductive st...
Predicted: Mathematical Induction
Confidence: 99.6%
----------------------------------------------------------------------
Proof: Consider two cases. Case 1: If x > 0, then |x| = x, so |x|/x = 1. Case 2: If x <...
Predicted: Proof by Cases
Confidence: 97.6%
----------------------------------------------------------------------


## Step 6: Save Model Files

Download these files and put them in your `backend/models/` folder.

In [11]:
import pickle
import json
import os

# Create models directory
os.makedirs('models', exist_ok=True)

# Save classifier
with open('models/classifier.pkl', 'wb') as f:
    pickle.dump(best_clf, f)
print("✅ Saved models/classifier.pkl")

# Save label encoder
with open('models/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)
print("✅ Saved models/label_encoder.pkl")

# Save config
config = {
    "model_name": "all-MiniLM-L6-v2",
    "embedding_dim": 384,
    "classifier_type": best_name,
    "num_classes": len(label_encoder.classes_),
    "classes": [TECHNIQUE_NAMES[int(c)] for c in label_encoder.classes_],
    "training_samples": len(train_texts),
    "cv_accuracy": float(best_score)
}
with open('models/config.json', 'w') as f:
    json.dump(config, f, indent=2)
print("✅ Saved models/config.json")

print("\n" + "=" * 50)
print("MODEL SUMMARY")
print("=" * 50)
for k, v in config.items():
    print(f"{k}: {v}")

✅ Saved models/classifier.pkl
✅ Saved models/label_encoder.pkl
✅ Saved models/config.json

MODEL SUMMARY
model_name: all-MiniLM-L6-v2
embedding_dim: 384
classifier_type: MLP (Neural Network)
num_classes: 7
classes: ['Direct Proof', 'Proof by Contradiction', 'Mathematical Induction', 'Contrapositive', 'Proof by Construction', 'Proof by Cases', 'Proof by Exhaustion']
training_samples: 66
cv_accuracy: 0.7736263736263737


In [ ]:
# Download the model files

print("📥 Downloading model files...")
print("   (Save these to your backend/models/ folder)\n")

files.download('models/classifier.pkl')
files.download('models/label_encoder.pkl')
files.download('models/config.json')

print("\n✅ Done!")

ModuleNotFoundError: No module named 'google'